In [1]:
from mxnet import nd,gluon,init,autograd, metric

from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets,transforms

import matplotlib.pyplot as plt
from time import time

In [2]:
net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=6,kernel_size=5,activation='relu'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Conv2D(channels=16,kernel_size=3,activation='relu'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Flatten(),
        nn.Dense(120,activation='relu'),
        nn.Dense(84,activation='relu'),
        nn.Dense(10)
    )
    
net.initialize(init=init.Xavier())

In [4]:
# Load the parameters
net.load_parameters('trained_params.params')

# Predict

In [5]:
# Validation Dataset

mainst_valid = datasets.FashionMNIST(train=False)

# Data Transformation

In [7]:
transformer_fn = transforms.Compose([transforms.ToTensor(),transforms.Normalize(0.13,0.31)])

# mainst_train = mainst_train.transform_first(transformer)

In [14]:
preds = []
images = []
labels = []
for idx in range(6):
    image,label = mainst_valid[idx]
    image = transformer_fn(image).expand_dims(axis=0)
    pred = net(image).argmax(axis=1)
    preds.append(pred.astype('int32').asscalar())
    labels.append(label)
    images.append(image)

In [15]:
labels

[0, 1, 2, 2, 3, 2]

In [16]:
preds

[0, 1, 2, 6, 4, 6]

# Validation dataloader

In [22]:
batch_size = 256

valid_data = gluon.data.DataLoader(mainst_valid.transform_first(transformer_fn),batch_size=batch_size,num_workers=4)

# Accuracy

In [23]:
valid_acc = metric.Accuracy()

# Validation Loop

In [24]:
for data,label in valid_data:
    output = net(data)
    valid_acc .update(label,output)

print("Validation Acc: %.3f"%(valid_acc.get()[1]))

Validation Acc: 0.891


# EXIT